# Chapter 22: Building Web Applications with ASP.NET Core

ASP.NET Core is a cross‑platform, high‑performance framework for building modern, cloud‑based, internet‑connected applications. Whether you're building a small REST API, a large e‑commerce site, or a real‑time web application, ASP.NET Core provides the tools and libraries you need.

In this chapter, you'll learn:

- The architecture of ASP.NET Core and its key components.
- The two main approaches: **Minimal APIs** (simple, lightweight) and **MVC** (Model‑View‑Controller) for more structured applications.
- How to define **routing** to map URLs to code.
- The **middleware pipeline** – how requests flow through your application.
- **Controllers** and **actions** as the core of MVC.
- **Model binding** and **validation** – how data from HTTP requests is automatically converted to .NET objects and validated.
- Using **Entity Framework Core** (EF Core) for data access: code‑first migrations, querying, and saving data.
- The deep integration of **dependency injection** in ASP.NET Core.
- A practical example: building a complete REST API for a simple note‑taking application.
- Best practices and common pitfalls.

By the end, you'll be able to build robust, maintainable web applications and APIs using the most popular framework in the .NET ecosystem.

---

## 22.1 Introduction to ASP.NET Core

ASP.NET Core is a redesign of the classic ASP.NET, built from the ground up to be modular, lightweight, and cross‑platform. It runs on Windows, Linux, and macOS, and can be hosted on IIS, Nginx, Apache, or as a self‑contained executable.

Key features:

- **Unified programming model** – you can build web UIs (Razor Pages, MVC) and web APIs using the same framework.
- **Dependency injection** is built in – no need for external containers (though you can plug them in).
- **Middleware** – a pipeline of components that handle requests and responses.
- **Cross‑platform** – develop and deploy anywhere.
- **High performance** – consistently ranked among the fastest web frameworks.
- **Open source** – community contributions and transparent development.

---

## 22.2 Project Templates and Structure

When you create a new ASP.NET Core project, you can choose from several templates: Empty, Web API, Web Application (MVC), etc.

### Creating a New Project

Using the .NET CLI:

```bash
dotnet new webapi -n MyApi
dotnet new mvc -n MyMvcApp
dotnet new web -n MinimalApp   # Minimal API template
```

### Project Structure (for Web API)

A typical Web API project has:

- `Controllers/` – folder for API controllers.
- `Program.cs` – the entry point and startup configuration.
- `appsettings.json` – configuration files.
- `Properties/launchSettings.json` – launch profiles for development.

In .NET 6 and later, `Program.cs` is simplified with top‑level statements and a minimal setup.

---

## 22.3 Minimal APIs vs. MVC

ASP.NET Core offers two primary ways to build HTTP APIs:

### Minimal APIs

Introduced in .NET 6, Minimal APIs are ideal for small microservices and simple endpoints. They use a concise syntax without the overhead of controllers.

```csharp
var builder = WebApplication.CreateBuilder(args);
var app = builder.Build();

app.MapGet("/", () => "Hello World!");
app.MapGet("/products", async (ProductDbContext db) => await db.Products.ToListAsync());
app.MapPost("/products", async (Product product, ProductDbContext db) =>
{
    db.Products.Add(product);
    await db.SaveChangesAsync();
    return Results.Created($"/products/{product.Id}", product);
});

app.Run();
```

Minimal APIs are great for simple scenarios, but as your application grows, you may want more structure.

### MVC (Model‑View‑Controller)

MVC is a mature pattern that separates your application into three main components:

- **Model** – represents the data and business logic.
- **View** – the UI layer (for web pages, not APIs).
- **Controller** – handles HTTP requests, works with the model, and returns responses.

For APIs, you typically use controllers without views (just returning data). Controllers provide features like model binding, validation, filters, and easier organization.

```csharp
[ApiController]
[Route("api/[controller]")]
public class ProductsController : ControllerBase
{
    private readonly ProductDbContext _context;

    public ProductsController(ProductDbContext context)
    {
        _context = context;
    }

    [HttpGet]
    public async Task<ActionResult<List<Product>>> GetAll() => await _context.Products.ToListAsync();

    [HttpGet("{id}")]
    public async Task<ActionResult<Product>> Get(int id)
    {
        var product = await _context.Products.FindAsync(id);
        if (product == null) return NotFound();
        return product;
    }

    [HttpPost]
    public async Task<ActionResult<Product>> Create(Product product)
    {
        _context.Products.Add(product);
        await _context.SaveChangesAsync();
        return CreatedAtAction(nameof(Get), new { id = product.Id }, product);
    }
}
```

Which one to choose? Minimal APIs for small projects or prototypes; MVC for larger, more structured applications where you need features like filters, model binding attributes, and better testability. Both are valid; you can even mix them.

---

## 22.4 Routing

Routing is how ASP.NET Core matches incoming HTTP requests to a specific handler (a controller action or a minimal API endpoint).

### Convention‑based Routing (MVC)

You can define routes using attributes or convention.

**Attribute Routing (preferred):**

```csharp
[Route("api/[controller]")]
public class ProductsController : ControllerBase
{
    [HttpGet]
    public IActionResult GetAll() { ... }

    [HttpGet("{id}")]
    public IActionResult Get(int id) { ... }
}
```

The `[controller]` token is replaced with the controller name (without "Controller"). So the route for `ProductsController` becomes `api/products`.

**Conventional Routing** (defined in `Program.cs`):

```csharp
app.MapControllerRoute(
    name: "default",
    pattern: "{controller=Home}/{action=Index}/{id?}");
```

### Routing in Minimal APIs

Routes are defined directly on the endpoint:

```csharp
app.MapGet("/products/{id}", async (int id, ProductDbContext db) => ...);
```

You can also define route constraints:

```csharp
app.MapGet("/products/{id:int}", ...); // only integers
```

---

## 22.5 Middleware Pipeline

Middleware are components that are assembled into a pipeline to handle requests and responses. Each middleware can perform work before and after the next middleware in the pipeline.

A simple middleware example:

```csharp
app.Use(async (context, next) =>
{
    // Before the next middleware
    Console.WriteLine("Request received");
    await next();
    // After the next middleware
    Console.WriteLine("Response sent");
});
```

Common built‑in middleware:

- `UseRouting()` – enables routing.
- `UseAuthentication()` – handles authentication.
- `UseAuthorization()` – handles authorization.
- `UseStaticFiles()` – serves static files.
- `UseEndpoints()` – maps endpoints (controllers or minimal APIs).

The order of middleware matters. For example, `UseAuthentication` must come before `UseAuthorization`.

---

## 22.6 Controllers and Actions

In MVC, a controller is a class that inherits from `ControllerBase` (for APIs) or `Controller` (for MVC with views). Actions are public methods on the controller that respond to HTTP requests.

### Returning Responses

Actions can return various types:

- `IActionResult` or `ActionResult<T>` – allows returning status codes, content, etc.
- Specific types – automatically wrapped in an `Ok` result.

```csharp
[HttpGet]
public ActionResult<List<Product>> GetAll() => _context.Products.ToList();

[HttpGet("{id}")]
public IActionResult Get(int id)
{
    var product = _context.Products.Find(id);
    if (product == null) return NotFound();
    return Ok(product);
}
```

Using `ActionResult<T>` gives you the benefits of both type safety and flexibility.

### Attributes for Actions

- `[HttpGet]`, `[HttpPost]`, `[HttpPut]`, `[HttpDelete]` – specify the HTTP method.
- `[Route]` – custom route for an action.
- `[Authorize]` – requires authentication.
- `[ApiController]` – enables API‑specific behaviors (automatic model validation, binding source inference, etc.).

---

## 22.7 Model Binding and Validation

**Model binding** automatically maps data from HTTP requests (query string, route data, form, body) to action method parameters or complex types.

```csharp
public IActionResult Get([FromQuery] int page, [FromRoute] int id) { ... }
[HttpPost]
public IActionResult Create([FromBody] Product product) { ... }
```

With `[ApiController]`, complex parameters are automatically bound from the body, and simple types from route/query. You can override with attributes.

**Validation** is performed via data annotations on model classes.

```csharp
public class Product
{
    public int Id { get; set; }
    [Required]
    [StringLength(100)]
    public string Name { get; set; }
    [Range(0.01, 10000)]
    public decimal Price { get; set; }
}
```

When `[ApiController]` is present, invalid models automatically trigger a `400 Bad Request` response with details.

---

## 22.8 Entity Framework Core (EF Core)

EF Core is the modern object‑relational mapper (ORM) for .NET. It allows you to work with a database using .NET objects, eliminating most of the data‑access plumbing.

### Setting Up EF Core

Add the necessary NuGet packages:

```bash
dotnet add package Microsoft.EntityFrameworkCore.SqlServer
dotnet add package Microsoft.EntityFrameworkCore.Tools  # for migrations
```

Define your `DbContext`:

```csharp
public class AppDbContext : DbContext
{
    public AppDbContext(DbContextOptions<AppDbContext> options) : base(options) { }
    public DbSet<Product> Products { get; set; }
}
```

Register it in `Program.cs`:

```csharp
builder.Services.AddDbContext<AppDbContext>(options =>
    options.UseSqlServer(builder.Configuration.GetConnectionString("DefaultConnection")));
```

### Code‑First Migrations

Migrations allow you to evolve your database schema as your model changes.

```bash
dotnet ef migrations add InitialCreate
dotnet ef database update
```

### Querying Data

```csharp
// LINQ queries
var products = await _context.Products.Where(p => p.Price > 100).ToListAsync();
var product = await _context.Products.FindAsync(id);
```

### Saving Data

```csharp
_context.Products.Add(new Product { Name = "New", Price = 10 });
await _context.SaveChangesAsync();
```

EF Core handles change tracking, concurrency, and much more.

---

## 22.9 Dependency Injection in ASP.NET Core

ASP.NET Core has a built‑in DI container. Services are registered in `Program.cs` and then injected into controllers, middleware, or other services.

```csharp
builder.Services.AddScoped<IProductService, ProductService>();
builder.Services.AddDbContext<AppDbContext>(...);
```

Controllers receive dependencies via constructor injection:

```csharp
public class ProductsController : ControllerBase
{
    private readonly IProductService _productService;
    public ProductsController(IProductService productService)
    {
        _productService = productService;
    }
}
```

This promotes loose coupling and testability.

---

## 22.10 Putting It All Together: A Notes API

Let's build a complete REST API for a note‑taking application. It will have:

- A `Note` model.
- An `AppDbContext` for SQLite (or SQL Server).
- A controller with CRUD operations.
- Dependency injection for the context.
- Model validation.

### Step 1: Create Project

```bash
dotnet new webapi -n NotesApi
cd NotesApi
dotnet add package Microsoft.EntityFrameworkCore.Sqlite
dotnet add package Microsoft.EntityFrameworkCore.Tools
```

### Step 2: Define Model

```csharp
// Models/Note.cs
using System.ComponentModel.DataAnnotations;

namespace NotesApi.Models;

public class Note
{
    public int Id { get; set; }

    [Required]
    [StringLength(200)]
    public string Title { get; set; } = string.Empty;

    [Required]
    public string Content { get; set; } = string.Empty;

    public DateTime CreatedAt { get; set; } = DateTime.UtcNow;
    public DateTime? UpdatedAt { get; set; }
}
```

### Step 3: Create DbContext

```csharp
// Data/AppDbContext.cs
using Microsoft.EntityFrameworkCore;
using NotesApi.Models;

namespace NotesApi.Data;

public class AppDbContext : DbContext
{
    public AppDbContext(DbContextOptions<AppDbContext> options) : base(options) { }
    public DbSet<Note> Notes { get; set; }
}
```

### Step 4: Configure Services and Database

In `Program.cs`:

```csharp
using Microsoft.EntityFrameworkCore;
using NotesApi.Data;

var builder = WebApplication.CreateBuilder(args);

// Add services to the container.
builder.Services.AddControllers();
builder.Services.AddDbContext<AppDbContext>(options =>
    options.UseSqlite("Data Source=notes.db"));

// Learn more about configuring Swagger/OpenAPI at https://aka.ms/aspnetcore/swashbuckle
builder.Services.AddEndpointsApiExplorer();
builder.Services.AddSwaggerGen();

var app = builder.Build();

// Configure the HTTP request pipeline.
if (app.Environment.IsDevelopment())
{
    app.UseSwagger();
    app.UseSwaggerUI();
}

app.UseHttpsRedirection();
app.UseAuthorization();
app.MapControllers();

// Ensure database is created
using (var scope = app.Services.CreateScope())
{
    var db = scope.ServiceProvider.GetRequiredService<AppDbContext>();
    db.Database.EnsureCreated();
}

app.Run();
```

### Step 5: Create Controller

```csharp
// Controllers/NotesController.cs
using Microsoft.AspNetCore.Mvc;
using Microsoft.EntityFrameworkCore;
using NotesApi.Data;
using NotesApi.Models;

namespace NotesApi.Controllers;

[Route("api/[controller]")]
[ApiController]
public class NotesController : ControllerBase
{
    private readonly AppDbContext _context;

    public NotesController(AppDbContext context)
    {
        _context = context;
    }

    // GET: api/notes
    [HttpGet]
    public async Task<ActionResult<IEnumerable<Note>>> GetNotes()
    {
        return await _context.Notes.ToListAsync();
    }

    // GET: api/notes/5
    [HttpGet("{id}")]
    public async Task<ActionResult<Note>> GetNote(int id)
    {
        var note = await _context.Notes.FindAsync(id);
        if (note == null) return NotFound();
        return note;
    }

    // POST: api/notes
    [HttpPost]
    public async Task<ActionResult<Note>> PostNote(Note note)
    {
        if (!ModelState.IsValid)
        {
            return BadRequest(ModelState);
        }

        note.CreatedAt = DateTime.UtcNow;
        _context.Notes.Add(note);
        await _context.SaveChangesAsync();

        return CreatedAtAction(nameof(GetNote), new { id = note.Id }, note);
    }

    // PUT: api/notes/5
    [HttpPut("{id}")]
    public async Task<IActionResult> PutNote(int id, Note note)
    {
        if (id != note.Id)
        {
            return BadRequest();
        }

        if (!ModelState.IsValid)
        {
            return BadRequest(ModelState);
        }

        note.UpdatedAt = DateTime.UtcNow;
        _context.Entry(note).State = EntityState.Modified;

        try
        {
            await _context.SaveChangesAsync();
        }
        catch (DbUpdateConcurrencyException)
        {
            if (!_context.Notes.Any(e => e.Id == id))
            {
                return NotFound();
            }
            throw;
        }

        return NoContent();
    }

    // DELETE: api/notes/5
    [HttpDelete("{id}")]
    public async Task<IActionResult> DeleteNote(int id)
    {
        var note = await _context.Notes.FindAsync(id);
        if (note == null)
        {
            return NotFound();
        }

        _context.Notes.Remove(note);
        await _context.SaveChangesAsync();

        return NoContent();
    }
}
```

### Step 6: Run and Test

Run the app:

```bash
dotnet run
```

Navigate to `https://localhost:5001/swagger` to see the Swagger UI and test the API.

You now have a fully functional REST API with validation, database persistence, and proper HTTP semantics.

---

## 22.11 Best Practices and Common Pitfalls

### 1. Use `[ApiController]` and `ControllerBase`

`[ApiController]` enables automatic model validation, binding source inference, and other helpful features.

### 2. Return Appropriate HTTP Status Codes

Use `Ok()`, `CreatedAtAction()`, `NoContent()`, `BadRequest()`, `NotFound()` consistently.

### 3. Validate Input

Use data annotations and check `ModelState.IsValid`. `[ApiController]` handles it automatically, but you can still customize.

### 4. Use Dependency Injection for All Services

Avoid `new`‑ing up dependencies. Let the container handle lifetimes.

### 5. Keep Controllers Thin

Controllers should only handle HTTP concerns. Move business logic to separate services.

### 6. Use Async/Await for I/O

Database calls, file operations, etc., should be async to avoid blocking threads.

### 7. Secure Your API

Use HTTPS, authentication (e.g., JWT), and authorization. Apply `[Authorize]` attributes as needed.

### 8. Version Your API

As your API evolves, consider versioning to avoid breaking existing clients.

### 9. Log and Monitor

Use `ILogger` to log important events, errors, and request information.

### 10. Write Integration Tests

Test your API endpoints to ensure they behave correctly.

---

## 22.12 Chapter Summary

In this chapter, you've built a solid foundation in ASP.NET Core web development:

- **Minimal APIs** and **MVC** – two ways to build APIs, each with its strengths.
- **Routing** – how URLs map to code.
- **Middleware pipeline** – the request/response processing chain.
- **Controllers** and **actions** – the core of MVC.
- **Model binding** and **validation** – automatically mapping and validating request data.
- **Entity Framework Core** – data access with migrations and LINQ.
- **Dependency injection** – integrated throughout the framework.
- A complete **Notes API** example demonstrating all these concepts.

ASP.NET Core is a vast topic, but this chapter gives you the essential knowledge to start building real‑world applications. As you continue, you'll explore areas like authentication, real‑time communication (SignalR), background tasks, and deployment.

In the next chapter, **Unit Testing for Quality Code**, you'll learn how to write automated tests to ensure your code works correctly and continues to do so as it evolves. You'll cover unit testing frameworks (xUnit, NUnit), mocking dependencies, and testing ASP.NET Core controllers.

**Exercises:**

1. Extend the Notes API by adding a `User` entity and associating notes with users. Implement authentication using JWT.
2. Add filtering and pagination to the `GET /api/notes` endpoint (e.g., `?page=1&pageSize=10`).
3. Write a middleware that logs the time taken for each request.
4. Convert the Notes API to use Minimal APIs instead of MVC. Compare the code size and organization.
5. Deploy the API to a cloud platform like Azure App Service or a local Docker container.

Now, get ready to ensure code quality with unit testing in Chapter 23!

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='21. dependency_injection.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='23. unit_testing_for_quality_code.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
